In [12]:
import pandas as pd
import awswrangler as awr
import openpyxl


In [13]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_baixadas.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_pagamentos = awr.athena.read_sql_query(query, database='silver')


In [14]:
df_pagamentos = df_pagamentos.drop_duplicates('ponteiro', keep='first')
df_pagamentos = df_pagamentos[df_pagamentos['data_baixa']>= pd.to_datetime('2025-01-31').date()]



In [15]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_inadimplentes = awr.athena.read_sql_query(query, database='silver')
df_inadimplentes = df_inadimplentes.drop_duplicates('ponteiro', keep='first')

estabelecendo a base de vencidos

In [16]:
today = pd.to_datetime('today').date()
month = today.month
df_inadimplentes['data_vencimento'] = pd.to_datetime(df_inadimplentes['data_vencimento'])


base_vencidos = df_inadimplentes[df_inadimplentes['data_vencimento']>= pd.to_datetime('2025-01-31')]
#base_vencidos = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.month == month]


In [17]:
base_vencidos['situacao'] = pd.NA
base_vencidos['data_atualizacao'] = pd.NA
base_vencidos.sort_values(by='data_vencimento', ascending=False)
base_vencidos.head()


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_11504\1941425127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos['situacao'] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_11504\1941425127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos['data_atualizacao'] = pd.NA


,codigo_cadastro,associado,grupo,matricula,conjunto,aplicacao_financeira,data_emissao,data_vencimento,valor_titulo,codigo,...,numero_boleto,nosso_numero,status,id_set,parent,cooperativa,placa,chassi,situacao,data_atualizacao
0,3159,VAML ENERGIA SOLAR LTDA,RECEITAS,20532,12449,Mensalidades/Benefícios,2024-11-22,2025-04-25,740.58,53,...,1164868,1164868-9,ATIVO,12449,20532,Stcoop,<NA>,<NA>,<NA>,<NA>
1,10760,TSUR TRANSPORTES LTDA,RECEITAS,20773,12651,Mensalidades/Benefícios,2024-12-10,2025-04-16,1106.84,254,...,1187704,1187704-8,ATIVO,12651,20773,Stcoop,<NA>,<NA>,<NA>,<NA>
2,4347,PADOVANI TRANSPORTES LTDA,RECEITAS,21926,9572,Mensalidades/Benefícios,2024-07-03,2025-05-05,881.98,1192,...,1007328,24/007328-0,ATIVO,9572,21926,Stcoop,<NA>,<NA>,<NA>,<NA>
3,10663,MEGA CONCRETO LTDA,RECEITAS,20573,13720,Mensalidades/Benefícios,2025-02-28,2025-06-02,2483.91,281,...,1182721,1182721-8,ATIVO,13720,20573,Stcoop,<NA>,<NA>,<NA>,<NA>
4,8461,MG LIMA TRANSPORTES - EIRELI,RECEITAS,9121,9497,Mensalidades/Benefícios,2024-07-01,2025-05-05,2499.00,343,...,1006970,24/006970-3,ATIVO,9497,9121,Stcoop,AXK5236,9BVAG20C2DE808835,<NA>,<NA>


acrescentando colunas importantes

In [18]:
df_pagamentos.loc[:, 'data_atualizacao'] = df_pagamentos['data_baixa']
df_pagamentos.loc[:, 'situacao']='PAGO'

base_vencidos.loc[:, 'data_atualizacao'] = base_vencidos['data_vencimento']
base_vencidos.loc[:, 'situacao']='INADIMPLENTE'

acrescentando colunas comuns e concatenando

In [19]:
colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "dias_atrasado",
    "data_atualizacao",
    "situacao"
]

for col in colunas_comuns:
    if col not in df_pagamentos.columns:
        df_pagamentos.loc[:, col] = pd.NA
    if col not in base_vencidos.columns:
        base_vencidos.loc[:, col] = pd.NA

df_historico = pd.concat([df_pagamentos, base_vencidos], ignore_index=True)
df_historico = df_historico[colunas_comuns]


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_11504\1360921929.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos.loc[:, col] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_11504\1360921929.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos.loc[:, col] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_11504\1360921929.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version

In [20]:
df_historico['data_atualizacao'] = pd.to_datetime(df_historico['data_atualizacao'])
df_historico['data_atualizacao'] = df_historico['data_atualizacao'].dt.date


passando a base para o excel

In [21]:
save_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia"
base_vencidos.to_excel(save_path + r"\historico_faturas.xlsx", index=False, sheet_name='historico_faturas')


In [22]:
df_pagamentos.head()

,cooperativa,aplicacao,ponteiro,numero_boleto,nosso_numero,matricula,conjunto,unidade,associado,valor_baixa,data_baixa,data_vencimento,dias_atraso,data_atualizacao,situacao,valor_titulo,data_emissao,dias_atrasado
327,Stcoop,166,213481,1181204,1181204-4,21718,13664,UNIDADE VILHENA,C. ZANQUETA,1266.54,2025-04-10,2025-04-10,0,2025-04-10,PAGO,<NA>,<NA>,<NA>
385,Stcoop,166,110371,1104390,1104390-9,4475,7518,UNIDADE VILHENA,DALTO&DALTO LTDA-EPP,2445.32,2025-02-03,2025-02-03,0,2025-02-03,PAGO,<NA>,<NA>,<NA>
396,Stcoop,166,133079,1124451,1124451-8,21916,8894,UNIDADE VILHENA,GEFSON LUIZ TASSI,525.34,2025-02-03,2025-02-03,0,2025-02-03,PAGO,<NA>,<NA>,<NA>
448,Stcoop,166,136394,1001931,24/001931-5,14826,9044,UNIDADE CASCAVEL,EDSON BUTURI,1380.69,2025-03-07,2025-03-10,3,2025-03-07,PAGO,<NA>,<NA>,<NA>
456,Stcoop,166,136395,1001928,24/001928-5,14826,9044,UNIDADE CASCAVEL,EDSON BUTURI,1380.69,2025-04-09,2025-04-10,1,2025-04-09,PAGO,<NA>,<NA>,<NA>


AGORA É:

OK criar a base (já tá criada) 

OK gerar a base com vencidos este MÊS

ACIMA PODE GERAR DE NOVO SÓ RODANDO ISSO AMANHÃ, ATÉ O CÓDIGO DE ATT FICAR PRONTO OU SEGUIR ABAIXO


depois gerar o código de atualização nessa ideia:

todo dia vai (pegar o excel) vai (gerar a base de inadimplencia de hoje) e vai (concatenar as duas)

(gerar uma lista com os ponteiros vencidos neste dia do mês)

(gerar a lista de baixados neste mês) e (filtrar pela lista de ponteiros vencidos)

(concatenar esses pagamentos filtrados)

(atualizar o excel)